**Plugins en Semantic Kernel**

Los plugins mejoran las capacidades de los modelos de lenguaje (LLM) mediante una ejecución modular, lo que te permite:

- **Ejecutar lógica determinística:**  
  Cada plugin contiene una lógica específica y bien definida para realizar tareas concretas.

- **Extender funcionalidades:**  
  Puedes ampliar las capacidades de los modelos de lenguaje integrándolos con diferentes herramientas y servicios.

- **Personalización a medida:**  
  Los plugins facilitan adaptar la funcionalidad según las necesidades de cada usuario o aplicación, ofreciendo flexibilidad y control a la hora de diseñar soluciones.

Los plugins te permiten enriquecer el LLM con funcionalidades adicionales de manera ordenada y modular, haciendo que la integración y extensión de servicios sea sencilla y eficaz.

In [6]:
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function, KernelArguments

# Definición del plugin de herramientas
class HerramientasPlugin:
    """Plugin de Semantic Kernel que extiende las capacidades del LLM de manera modular.
    
    Este plugin contiene funciones determinísticas para actividades específicas.
    """

    @kernel_function(name="saludar")
    async def saludar(self, nombre: str) -> str:
        """Devuelve un saludo personalizado."""
        return f"¡Hola, {nombre}! Bienvenido al sistema modular."

    @kernel_function(name="convertir_mayusculas")
    async def convertir_mayusculas(self, texto: str) -> str:
        """Convierte el mensaje a mayúsculas."""
        return texto.upper()


# Crear la instancia del kernel
kernel = Kernel()

# Registrar el plugin "herramientas" en el kernel
plugin = HerramientasPlugin()
kernel.add_plugin(plugin, plugin_name="herramientas")


# Ejemplo de invocación de funciones del plugin
async def main():
    # Invocar la función 'saludar'
    args_saludar = KernelArguments(nombre="Juan")
    resultado_saludo = await kernel.invoke(function_name="saludar", plugin_name="herramientas", arguments=args_saludar)
    print(resultado_saludo)  # Salida: ¡Hola, Juan! Bienvenido al sistema modular.

    # Invocar la función 'convertir_mayusculas'
    args_mayusculas = KernelArguments(texto="hola mundo")
    resultado_mayusculas = await kernel.invoke(function_name="convertir_mayusculas", plugin_name="herramientas", arguments=args_mayusculas)
    print(resultado_mayusculas)  # Salida: HOLA MUNDO


await main()

¡Hola, Juan! Bienvenido al sistema modular.
HOLA MUNDO


**Interacción en Tiempo Real y Condiciones Definidas**

- **Integración con APIs Externas:**  
  Los plugins permiten conectar de forma sencilla con diferentes APIs y acceder a datos en tiempo real.

- **Ejecución Condicional y Segura:**  
  Garantizan el disparo de tareas específicas cuando se cumplen condiciones claramente definidas, asegurando una ejecución precisa.

Estos complementos facilitan la integración de fuentes externas y la ejecución de acciones bajo criterios específicos, haciendo que tu aplicación sea más dinámica y confiable.

In [9]:
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function, KernelArguments
import requests

class RealTimePlugin:
    """
    Plugin para Interacción en Tiempo Real y Condiciones Definidas.
    
    - **Integración con APIs Externas:**  
      Conecta con servicios externos para obtener datos en tiempo real, por ejemplo la temperatura actual de una ciudad.
    
    - **Ejecución Condicional y Segura:**  
      Ejecuta tareas específicas cuando se cumplen condiciones predefinidas. En este caso, evalúa si la temperatura supera un umbral.
    """

    @kernel_function(name="obtener_temperatura")
    async def obtener_temperatura(self, ciudad: str) -> float:
        """Consulta una API externa para obtener la temperatura actual de una ciudad."""
        # Nota: Para una integración real, descomenta y configura la siguiente línea.
        # response = requests.get(f"https://api.weatherapi.com/v1/current.json?key=YOUR_API_KEY&q={ciudad}")
        # data = response.json()
        # return data["current"]["temp_c"]
        # Para este ejemplo, se simula una temperatura en Celsius
        return 30.5

    @kernel_function(name="evaluar_condicion_temperatura")
    async def evaluar_condicion_temperatura(self, ciudad: str, umbral: float) -> str:
        """Evalúa si la temperatura actual supera el umbral especificado y dispara una acción."""
        temperatura = await self.obtener_temperatura(ciudad=ciudad)
        if temperatura > umbral:
            return (f"La temperatura en {ciudad} es de {temperatura}°C, "
                    f"superando el umbral de {umbral}°C. Se activa la acción programada.")
        else:
            return (f"La temperatura en {ciudad} es de {temperatura}°C, "
                    f"por debajo del umbral de {umbral}°C. No se ejecuta acción alguna.")

# Crear la instancia del kernel y registrar el plugin
kernel = Kernel()
api_plugin = RealTimePlugin()
kernel.add_plugin(api_plugin, plugin_name="real_time")

# Ejemplo de invocación de funciones del plugin
async def main():
    # Invocar función para obtener la temperatura en tiempo real
    args_temp = KernelArguments(ciudad="Madrid")
    temperatura = await kernel.invoke(
        function_name="obtener_temperatura", 
        plugin_name="real_time", 
        arguments=args_temp
    )
    print(f"Temperatura actual en Madrid: {temperatura}°C")
    
    # Invocar función para evaluar la condición y disparar la acción si es necesario
    args_condicion = KernelArguments(ciudad="Madrid", umbral=28.0)
    resultado = await kernel.invoke(
        function_name="evaluar_condicion_temperatura", 
        plugin_name="real_time", 
        arguments=args_condicion
    )
    print(resultado)


await main()

Temperatura actual en Madrid: 30.5°C
La temperatura en Madrid es de 30.5°C, superando el umbral de 28.0°C. Se activa la acción programada.


---

**Arquitectura Modular Respetuosa de Principios SOLID**

- **Principio de Abierto/Cerrado:**  
  Cada plugin está diseñado para que sea ampliable sin modificar su código base, permitiendo futuras mejoras sin interrumpir la funcionalidad existente.

- **Principio de Responsabilidad Única:**  
  Cada plugin cumple una única función específica, lo que garantiza que su lógica sea clara, bien definida y fácil de mantener.

En conjunto, estos principios aseguran que los plugins sean **modulares** y **escalables**, facilitando la integración y evolución de tus soluciones de manera ordenada y eficaz.

---

In [11]:
import abc
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function, KernelArguments


# Base class for all plugins using the Single Responsibility Principle.
class BasePlugin(abc.ABC):
    """Base plugin class that defines the interface for plugins."""

    @abc.abstractmethod
    def register_functions(self, kernel: Kernel):
        """Register plugin functions in the given kernel."""
        pass


# Plugin especializado en saludos.
class SaludoPlugin(BasePlugin):
    """
    Plugin responsable de generar saludos personalizados.
    
    Cumple con el Principio de Responsabilidad Única y se puede extender sin
    modificar su código base (Principio de Abierto/Cerrado).
    """

    @kernel_function(name="saludar")
    async def saludar(self, nombre: str) -> str:
        """Devuelve un saludo personalizado para el usuario."""
        return f"¡Hola, {nombre}! Bienvenido a nuestro sistema modular."

    def register_functions(self, kernel: Kernel):
        kernel.add_plugin(self, plugin_name="saludo")


# Plugin especializado en convertir texto a mayúsculas.
class ConversionPlugin(BasePlugin):
    """
    Plugin responsable de convertir mensajes a mayúsculas.
    
    Cada plugin cumple una única función, facilitando una lógica clara y mantenible, 
    y se puede ampliar sin alterar la implementación existente.
    """

    @kernel_function(name="convertir_mayusculas")
    async def convertir_mayusculas(self, texto: str) -> str:
        """Convierte el mensaje a mayúsculas."""
        return texto.upper()

    def register_functions(self, kernel: Kernel):
        kernel.add_plugin(self, plugin_name="conversion")


# Crear la instancia del kernel
kernel = Kernel()

# Registrar los plugins utilizando sus propios métodos de registro
saludo_plugin = SaludoPlugin()
saludo_plugin.register_functions(kernel)

conversion_plugin = ConversionPlugin()
conversion_plugin.register_functions(kernel)


# Ejemplo de invocación de funciones de cada plugin
async def main():
    # Invocar la función 'saludar' del plugin "saludo"
    args_saludo = KernelArguments(nombre="Ana")
    resultado_saludo = await kernel.invoke(
        function_name="saludar",
        plugin_name="saludo",
        arguments=args_saludo
    )
    print("Resultado saludo:", resultado_saludo)

    # Invocar la función 'convertir_mayusculas' del plugin "conversion"
    args_conversion = KernelArguments(texto="hola mundo")
    resultado_conversion = await kernel.invoke(
        function_name="convertir_mayusculas",
        plugin_name="conversion",
        arguments=args_conversion
    )
    print("Resultado conversión:", resultado_conversion)


await main()

Resultado saludo: ¡Hola, Ana! Bienvenido a nuestro sistema modular.
Resultado conversión: HOLA MUNDO


---

**Plugin de Memoria de Texto**  
- **Funcionalidad:** Almacena datos estructurados para mantener una *conciencia del contexto* a lo largo de la conversación.

---

**Plugin de Resumen de Conversación**  
- **Funcionalidad:** Optimiza la gestión del diálogo, ayudando a sintetizar y organizar la información para facilitar el flujo de la conversación.

---

**Plugin HTTP**  
- **Funcionalidad:** Recupera datos en tiempo real a través de llamadas a APIs REST, conectando tu aplicación con fuentes externas de manera eficiente.

---

In [14]:
from semantic_kernel import Kernel
from semantic_kernel.functions import KernelArguments
from semantic_kernel.core_plugins import (
    TextMemoryPlugin,
    ConversationSummaryPlugin,
    HttpPlugin,
)

# Crear la instancia del kernel
kernel = Kernel()

# Registrar los plugins utilizando sus propios métodos de registro
text_memory_plugin = TextMemoryPlugin(memory={})
kernel.add_plugin(text_memory_plugin, plugin_name="text_memory")

conversation_summary_plugin = ConversationSummaryPlugin()
kernel.add_plugin(conversation_summary_plugin, plugin_name="conversation_summary")

http_plugin = HttpPlugin()
kernel.add_plugin(http_plugin, plugin_name="http")


# Ejemplo de invocación de funciones de cada plugin
async def main():
    # Invocar la función 'almacenar_dato' del plugin "text_memory"
    args_almacenar = KernelArguments(clave="usuario", valor="Ana")
    resultado_almacenar = await kernel.invoke(
        function_name="almacenar_dato",
        plugin_name="text_memory",
        arguments=args_almacenar
    )
    print("Resultado almacenar:", resultado_almacenar)

    # Invocar la función 'recuperar_dato' del plugin "text_memory"
    args_recuperar = KernelArguments(clave="usuario")
    resultado_recuperar = await kernel.invoke(
        function_name="recuperar_dato",
        plugin_name="text_memory",
        arguments=args_recuperar
    )
    print("Resultado recuperar:", resultado_recuperar)

    # Invocar la función 'resumir_conversacion' del plugin "conversation_summary"
    args_resumir = KernelArguments(conversacion="Esta es una conversación larga que necesita ser resumida.")
    resultado_resumir = await kernel.invoke(
        function_name="resumir_conversacion",
        plugin_name="conversation_summary",
        arguments=args_resumir
    )
    print("Resultado resumen:", resultado_resumir)

    # Invocar la función 'realizar_llamada_api' del plugin "http"
    args_http = KernelArguments(url="https://api.github.com")
    resultado_http = await kernel.invoke(
        function_name="realizar_llamada_api",
        plugin_name="http",
        arguments=args_http
    )
    print("Resultado HTTP:", resultado_http)


await main()

TypeError: Can't instantiate abstract class SemanticTextMemoryBase without an implementation for abstract methods 'get', 'get_collections', 'save_information', 'save_reference', 'search'